In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

# Model

In [2]:
save_dir = './checkpoints'
logdir = './logs'

class CNN:
    def __init__(self):
        input = Input(shape=(200,200,3))
        conv1 = Conv2D(140,(3,3),activation="relu")(input)
        conv2 = Conv2D(130,(3,3),activation="relu")(conv1)
        batch1 = BatchNormalization()(conv2)
        pool3 = MaxPool2D((2,2))(batch1)
        conv3 = Conv2D(120,(3,3),activation="relu")(pool3)
        batch2 = BatchNormalization()(conv3)
        pool4 = MaxPool2D((2,2))(batch2)
        flt = Flatten()(pool4)

        #age
        age_l = Dense(128,activation="relu")(flt)
        age_l = Dense(64,activation="relu")(age_l)
        age_l = Dense(32,activation="relu")(age_l)
        age_l = Dense(1,activation="relu")(age_l)

        #gender
        gender_l = Dense(128,activation="relu")(flt)
        gender_l = Dense(80,activation="relu")(gender_l)
        gender_l = Dense(64,activation="relu")(gender_l)
        gender_l = Dense(32,activation="relu")(gender_l)
        gender_l = Dropout(0.5)(gender_l)
        gender_l = Dense(2,activation="softmax")(gender_l)

        self.model = Model(inputs = input,outputs = [age_l,gender_l])
        self.model.compile(optimizer="adam",loss=["mse","sparse_categorical_crossentropy"],metrics=['mae','accuracy'])

    def train(self, x_train, y_train, y_train_2, x_test, y_test, y_test_2,batch_size=8, epochs=50):
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(save_dir)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

        save = self.model.fit(
                        x_train,[y_train,y_train_2],
                        batch_size=batch_size,
                        validation_data=(x_test,[y_test,y_test_2]),
                        callbacks=[checkpoint_callback , tensorboard_callback, early_stopping_callback],
                        epochs=epochs)
        return save

# Processing Dataset

In [3]:
path = "./UTKDataset/UTKFace"
pixels = []
age = []
gender = []

for img in os.listdir(path):
  ages = img.split("_")[0]
  genders = img.split("_")[1]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  pixels.append(np.array(img))
  age.append(np.array(ages))
  gender.append(np.array(genders))
age = np.array(age,dtype=np.int64)
pixels = np.array(pixels)
gender = np.array(gender,np.uint64)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(pixels,age,test_size=0.3,random_state=100)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(pixels,gender,test_size=0.3,random_state=100)

# Training loop

In [5]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:
model = CNN()

save = model.train(x_train, y_train, y_train_2, x_test, y_test, y_test_2)
model.save('model.h5')

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

# Evaluation

# Tensorflow Lite